In [1]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

#top 4 features로 model 만들어서 accuracy구하기
import pandas as pd
import random
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
tf.random.set_seed(777) #하이퍼파라미터 튜닝을 위해 실행시 마다 변수가 같은 초기값 가지게 하기


   
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x
    
    
train_x, train_y = get_x_y(train)
test_x = get_x_y(test)

class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]



snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)
    
    
    
train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)

for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

train_y = to_categorical(train_y)

    
scaler = MaxAbsScaler()
train_x[:] = scaler.fit_transform(train_x[:])
test_x[:] = scaler.fit_transform(test_x[:])   


input = tf.keras.layers.Input(shape=(19,))
net = tf.keras.layers.Dense(units=32, activation='relu')(input)
net = tf.keras.layers.Dense(units=32, activation='relu')(net)
net = tf.keras.layers.Dense(units=14, activation='sigmoid')(net)
model = tf.keras.models.Model(input, net)

##########모델 학습

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_x, train_y, epochs=50)

preds = model.predict(test_x)
print('Done.')
submit = pd.read_csv('./sample_submission.csv')
submit['class'] = class_le.inverse_transform(preds)
submit.to_csv('./answer/tensorflow.csv', index=False)

Epoch 1/50


ValueError: in user code:

    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\k\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\k\anaconda3\lib

In [ ]:
train_y = y_train.apply(lambda ys: [y for y in ys.split(',')])

mlb = MultiLabelBinarizer()
mlb.fit(train_y)

train_y = mlb.transform(train_y)